In [34]:
import json
import requests
import flickrapi
import pandas as pd
from tqdm import tqdm
from vincenty import vincenty

In [35]:
api_key = u'b766ae533b2843f748b174f53885eee4'
api_secret = u'ac3884caa9690723'

In [36]:
flickr = flickrapi.FlickrAPI(api_key, api_secret)

In [37]:
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='json')
photos = flickr.photos.search(per_page='250', bbox='21.402542, 60.562697, 28.664383, 69.766582', has_geo=1)
parsed = json.loads(photos.decode('utf-8'))

id = parsed['photos']['photo'][0]['id']
geo = flickr.do_flickr_call('flickr.photos.geo.getLocation', photo_id=id)
print(json.loads(geo.decode('utf-8')))


{'photo': {'id': '50966372106', 'location': {'latitude': '63.447933', 'longitude': '26.673366', 'accuracy': '16', 'context': '0', 'locality': {'_content': 'Pielavesi'}, 'county': {'_content': 'Ylä-Savo'}, 'region': {'_content': 'North Savo'}, 'country': {'_content': 'Finland'}, 'neighbourhood': {'_content': 'Nousiala'}}}, 'stat': 'ok'}


In [40]:
photos_set = []
for i in tqdm(range(1,55)):
    res = flickr.photos.search(per_page='250', bbox='20.410, 59.800, 30.920, 69.910', has_geo=1)
    parsed = json.loads(photos.decode('utf-8'))
    for p in parsed['photos']['photo']:
        photos_set.append(p)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:23<00:00,  2.34it/s]


In [41]:
len(photos_set)

9342

In [42]:
photos_set[0]

{'id': '50966372106',
 'owner': '18215332@N00',
 'secret': '7e8144f8b3',
 'server': '65535',
 'farm': 66,
 'title': 'Flowing',
 'ispublic': 1,
 'isfriend': 0,
 'isfamily': 0}

In [43]:
# photo_id, lat, lon, name, locality, neighbourhood
photo_ids = []
lats = []
longs = []
names = []
localities = []
neighbourhoods = []
for p in tqdm(photos_set):
    photo_id = p['id']
    geo = flickr.do_flickr_call('flickr.photos.geo.getLocation', photo_id=photo_id)
    parsed = json.loads(geo.decode('utf-8'))
    ## construct arrays of data
    photo_ids.append(photo_id)
    names.append(p['title'])
    lats.append(parsed['photo']['location']['latitude'])
    longs.append(parsed['photo']['location']['longitude'])
    localities.append(parsed['photo']['location']['locality']['_content'])
    neighbourhoods.append(parsed['photo']['location']['neighbourhood']['_content'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9342/9342 [34:13<00:00,  4.55it/s]


## Data collection strategy:
We collect geographical information on all pictures taken in Finland inside the BBOX 21.402542, 60.562697, 28.664383, 69.766582.
Then, we test various clusterization algorithms to create ~500 nodes where each node represents a location.
For each location we can find ot the number of pictures taken, the most common names and tags. 
The edge is represented by distance calculated with Vincenty formula. 

In [33]:
df = pd.DataFrame({'id': photo_ids,
                  'name': names,
                  'lat': lats,
                  'lon': longs,
                  'locality': localities,
                  'neighbourhood': neighbourhoods})
df.to_csv('photoset.csv')